In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import logging

In [2]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [3]:
base_path = './data'
feature_path = './feature'

In [4]:
# 加载邀请回答数据
train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
train.columns = ['qid', 'uid', 'dt', 'label']
del train['dt']
logging.info("invite %s", train.shape)

test = pd.read_csv(f'{base_path}/invite_info_evaluate_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']
sub = test.copy()
sub_size = len(sub)
del test['dt']
logging.info("test %s", test.shape)



[2019-12-06 02:57:01,776] INFO in <ipython-input-4-f5fcce844f4e>: invite (9489162, 3)
[2019-12-06 02:57:03,272] INFO in <ipython-input-4-f5fcce844f4e>: test (1141683, 2)


In [5]:
# 加载 kfold feature
t1 = pd.read_csv(f'{feature_path}/train_kfold_feature.txt', sep='\t')
train = pd.concat([train, t1], axis=1)

t1 = pd.read_csv(f'{feature_path}/test_kfold_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)

In [6]:
# 加载 user 过去两个月的回答统计特征（除当条记录）
# t1 = pd.read_csv(f'{feature_path}/train_ua_feature.txt', sep='\t')
# train = pd.concat([train, t1], axis=1)

# t1 = pd.read_csv(f'{feature_path}/test_ua_feature.txt', sep='\t')
# test = pd.concat([test, t1], axis=1)

In [7]:
# 加载 invete feature 1
t1 = pd.read_csv(f'{feature_path}/train_invite_feature.txt', sep='\t')
train = pd.concat([train, t1], axis=1)

t1 = pd.read_csv(f'{feature_path}/test_invite_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)

In [8]:
# 加载 invete feature 2
t1 = pd.read_csv(f'{feature_path}/train_invite_feature_2.txt', sep='\t')
train = pd.concat([train, t1], axis=1)

t1 = pd.read_csv(f'{feature_path}/test_invite_feature_2.txt', sep='\t')
test = pd.concat([test, t1], axis=1)

In [9]:
# 加载 topic feature
t1 = pd.read_csv(f'{feature_path}/train_topic_feature.txt', sep='\t')
train = pd.concat([train, t1], axis=1)

t1 = pd.read_csv(f'{feature_path}/test_topic_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)

In [10]:
# 加载用户
user = pd.read_csv(f'{base_path}/member_info_0926.txt', header=None, sep='\t')
user.columns = ['uid', 'gender', 'freq',
                'uf_b1', 'uf_b2','uf_b3', 'uf_b4', 'uf_b5', 
                'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5', 
                'score', 'follow_topic', 'inter_topic']

del user['follow_topic'], user['inter_topic']
logging.info("user %s", user.shape)

unq = user.nunique()
logging.info("user unq %s", unq)

for x in unq[unq == 1].index:
    del user[x]
    logging.info('del unq==1 %s', x)

t = user.dtypes
cats = [x for x in t[t == 'object'].index if x not in ['follow_topic', 'inter_topic', 'uid']]
logging.info("user cat %s", cats)

for d in cats:
    lb = LabelEncoder()
    user[d] = lb.fit_transform(user[d])
    logging.info('encode %s', d)
    
q_lb = LabelEncoder()
q_lb.fit(list(train['qid'].astype(str).values) + list(test['qid'].astype(str).values))
train['qid_enc'] = q_lb.transform(train['qid'])
test['qid_enc'] = q_lb.transform(test['qid'])

u_lb = LabelEncoder()
u_lb.fit(user['uid'])
train['uid_enc'] = u_lb.transform(train['uid'])
test['uid_enc'] = u_lb.transform(test['uid'])

# merge user
train = pd.merge(train, user, on='uid', how='left')
test = pd.merge(test, user, on='uid', how='left')
logging.info("train shape %s, test shape %s", train.shape, test.shape)

[2019-12-06 03:01:03,831] INFO in <ipython-input-10-af285af762e4>: user (1931654, 14)
[2019-12-06 03:01:07,199] INFO in <ipython-input-10-af285af762e4>: user unq uid       1931654
gender          3
freq            5
uf_b1           2
uf_b2           2
uf_b3           2
uf_b4           2
uf_b5           2
uf_c1        2561
uf_c2         291
uf_c3         428
uf_c4        1556
uf_c5           2
score         732
dtype: int64
[2019-12-06 03:01:07,205] INFO in <ipython-input-10-af285af762e4>: user cat ['gender', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
[2019-12-06 03:01:08,066] INFO in <ipython-input-10-af285af762e4>: encode gender
[2019-12-06 03:01:08,954] INFO in <ipython-input-10-af285af762e4>: encode freq
[2019-12-06 03:01:09,810] INFO in <ipython-input-10-af285af762e4>: encode uf_c1
[2019-12-06 03:01:10,647] INFO in <ipython-input-10-af285af762e4>: encode uf_c2
[2019-12-06 03:01:11,461] INFO in <ipython-input-10-af285af762e4>: encode uf_c3
[2019-12-06 03:01:12,247] INFO in

In [11]:
data = pd.concat((train, test), axis=0, sort=True)
len_train = len(train)
del train

In [12]:
data

,day,diff_iq_day,diff_iq_hour,freq,gender,hour,intersection_ft_count,intersection_it_count,intersection_it_score,label,...,uid_hour_count,uid_hour_max,uid_hour_mean,uid_hour_median,uid_hour_min,uid_hour_std,uid_week_count,uid_week_mean,uid_week_median,uid_week_std
0,3865,4,95,4,2,22,1,0,0.000000,0.0,...,2,23,20.400000,22.0,12,4.722288,2,2.200000,2.0,1.643168
1,3844,21,495,1,2,11,0,0,0.000000,0.0,...,2,14,9.875000,9.5,7,3.044316,2,3.625000,4.0,1.922610
2,3862,1,24,4,2,15,0,0,0.000000,0.0,...,4,19,13.071428,13.0,7,3.561855,3,3.357143,3.5,1.736803
3,3849,2,37,0,2,11,0,1,1.066367,0.0,...,2,20,8.714286,8.0,0,6.421689,1,3.428571,3.0,1.718249
4,3867,20,469,1,2,4,0,0,0.000000,0.0,...,2,23,14.900000,16.0,4,6.740425,2,3.000000,3.0,2.108185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141678,3869,0,0,4,2,20,0,0,0.000000,NaN,...,1,23,19.777779,21.0,13,3.419714,1,3.333333,3.0,2.000000
1141679,3872,0,1,1,2,21,0,0,0.000000,NaN,...,1,23,21.000000,21.5,18,2.160247,1,4.000000,4.5,2.160247
1141680,3871,1,27,0,2,15,1,0,0.000000,NaN,...,1,21,16.916666,18.0,11,3.315483,2,3.083333,3.0,2.020726
1141681,3871,0,8,4,2,8,0,0,0.000000,NaN,...,10,23,11.421053,8.0,7,6.176261,1,3.000000,3.0,1.795055


In [13]:
# count 特征
count_fea = ['uid_enc', 'qid_enc', 'gender', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
for feat in count_fea:
    col_name = '{}_count'.format(feat)
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data.loc[data[col_name] < 2, feat] = -1
    data[feat] += 1
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data[col_name] = (data[col_name] - data[col_name].min()) / (data[col_name].max() - data[col_name].min())

In [14]:
data['wk'] = data['day'] % 7

In [15]:
drop_feat = ('label', 'uid', 'qid', 'dt', 'day') 
# drop_feat += ('q_ans_kfold_count', 'q_diff_qa_days_max', 'q_diff_qa_days_mean', 'q_diff_qa_days_sum', 
#               'q_has_img_max', 'q_has_img_mean', 'q_has_img_sum', 'q_has_video_max', 'q_has_video_mean', 
#               'q_has_video_sum','q_is_dest_max', 'q_is_dest_mean', 'q_is_dest_sum', 'q_is_good_max', 
#               'q_is_good_mean', 'q_is_good_sum', 'q_is_rec_max', 'q_is_rec_mean', 'q_is_rec_sum', 
#               'q_reci_cheer_max', 'q_reci_cheer_mean', 'q_reci_cheer_sum', 'q_reci_comment_max', 
#               'q_reci_comment_mean', 'q_reci_comment_sum', 'q_reci_dis_max', 'q_reci_dis_mean', 
#               'q_reci_dis_sum', 'q_reci_mark_max', 'q_reci_mark_mean', 'q_reci_mark_sum', 'q_reci_no_help_max',
#               'q_reci_no_help_mean', 'q_reci_no_help_sum', 'q_reci_tks_max', 'q_reci_tks_mean', 
#               'q_reci_tks_sum', 'q_reci_uncheer_max', 'q_reci_uncheer_mean', 'q_reci_uncheer_sum', 
#               'q_reci_xxx_max', 'q_reci_xxx_mean', 'q_reci_xxx_sum', 'q_word_count_max', 'q_word_count_mean', 
#               'q_word_count_sum')
# drop_feat += ('u_ans_kfold_count', 'u_diff_qa_days_max', 'u_diff_qa_days_mean', 'u_diff_qa_days_sum', 
#               'u_has_img_max', 'u_has_img_mean', 'u_has_img_sum', 'u_has_video_max', 'u_has_video_mean', 
#               'u_has_video_sum', 'u_is_dest_max', 'u_is_dest_mean', 'u_is_dest_sum', 'u_is_good_max', 
#               'u_is_good_mean', 'u_is_good_sum', 'u_is_rec_max', 'u_is_rec_mean', 'u_is_rec_sum', 'u_reci_cheer_max', 
#               'u_reci_cheer_mean', 'u_reci_cheer_sum', 'u_reci_comment_max', 'u_reci_comment_mean',
#               'u_reci_comment_sum', 'u_reci_dis_max', 'u_reci_dis_mean', 'u_reci_dis_sum', 'u_reci_mark_max', 
#               'u_reci_mark_mean', 'u_reci_mark_sum', 'u_reci_no_help_max', 'u_reci_no_help_mean', 
#               'u_reci_no_help_sum', 'u_reci_tks_max', 'u_reci_tks_mean', 'u_reci_tks_sum', 'u_reci_uncheer_max', 
#               'u_reci_uncheer_mean', 'u_reci_uncheer_sum', 'u_reci_xxx_max', 'u_reci_xxx_mean', 'u_reci_xxx_sum', 
#               'u_word_count_max', 'u_word_count_mean', 'u_word_count_sum')
# drop_feat += ('u_total_answer',)
feature_cols = [x for x in data.columns if x not in drop_feat]
# feature_cols

In [ ]:
logging.info("feature size %s", len(feature_cols))

X_train_all = data.iloc[:len_train][feature_cols]
y_train_all = data.iloc[:len_train]['label']
X_test = data.iloc[len_train:]
assert len(X_test) == sub_size

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break

X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
del X_train_all

logging.info("train shape %s, val shape %s, test shape %s", X_train.shape, X_val.shape, X_test.shape)

model_lgb = LGBMClassifier(n_estimators=2000, n_jobs=-1, objective='binary', seed=1000, silent=True)
model_lgb.fit(X_train, y_train,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=50)



[2019-12-06 03:05:11,125] INFO in <ipython-input-16-727117a4265e>: feature size 196
[2019-12-06 03:06:08,910] INFO in <ipython-input-16-727117a4265e>: train shape (7591329, 196), val shape (1897833, 196), test shape (1141683, 200)


[1]	valid_0's auc: 0.794436	valid_0's binary_logloss: 0.447296
Training until validation scores don't improve for 50 rounds
[2]	valid_0's auc: 0.815141	valid_0's binary_logloss: 0.432122
[3]	valid_0's auc: 0.821947	valid_0's binary_logloss: 0.419924
[4]	valid_0's auc: 0.825224	valid_0's binary_logloss: 0.410106
[5]	valid_0's auc: 0.829625	valid_0's binary_logloss: 0.401292
[6]	valid_0's auc: 0.831878	valid_0's binary_logloss: 0.394254
[7]	valid_0's auc: 0.834176	valid_0's binary_logloss: 0.387798
[8]	valid_0's auc: 0.835988	valid_0's binary_logloss: 0.38233
[9]	valid_0's auc: 0.838468	valid_0's binary_logloss: 0.377353
[10]	valid_0's auc: 0.840219	valid_0's binary_logloss: 0.373143
[11]	valid_0's auc: 0.84083	valid_0's binary_logloss: 0.369548
[12]	valid_0's auc: 0.842934	valid_0's binary_logloss: 0.366086
[13]	valid_0's auc: 0.844553	valid_0's binary_logloss: 0.36309
[14]	valid_0's auc: 0.846388	valid_0's binary_logloss: 0.359838
[15]	valid_0's auc: 0.847351	valid_0's binary_logloss: 

[129]	valid_0's auc: 0.883685	valid_0's binary_logloss: 0.30257
[130]	valid_0's auc: 0.883767	valid_0's binary_logloss: 0.302469
[131]	valid_0's auc: 0.883845	valid_0's binary_logloss: 0.302349
[132]	valid_0's auc: 0.883904	valid_0's binary_logloss: 0.302272
[133]	valid_0's auc: 0.883982	valid_0's binary_logloss: 0.302162
[134]	valid_0's auc: 0.884088	valid_0's binary_logloss: 0.302049
[135]	valid_0's auc: 0.884236	valid_0's binary_logloss: 0.301819
[136]	valid_0's auc: 0.884319	valid_0's binary_logloss: 0.30172
[137]	valid_0's auc: 0.884398	valid_0's binary_logloss: 0.30162
[138]	valid_0's auc: 0.884516	valid_0's binary_logloss: 0.30146
[139]	valid_0's auc: 0.884596	valid_0's binary_logloss: 0.301371
[140]	valid_0's auc: 0.884693	valid_0's binary_logloss: 0.301248
[141]	valid_0's auc: 0.884784	valid_0's binary_logloss: 0.301129
[142]	valid_0's auc: 0.88486	valid_0's binary_logloss: 0.301029
[143]	valid_0's auc: 0.884934	valid_0's binary_logloss: 0.30094
[144]	valid_0's auc: 0.884987	v

In [19]:
# sub = pd.read_csv(f'{base_path}/invite_info_evaluate_0926.txt', sep='\t', header=None)
# sub.columns = ['qid', 'uid', 'dt']
sub['label'] = model_lgb.predict_proba(X_test[feature_cols])[:, 1]

In [20]:
sub.to_csv('./result/2000_add_invite.txt', index=None, header=None, sep='\t')

In [21]:
fi = pd.DataFrame({'feature': feature_cols, 'imp': model_lgb.feature_importances_})
fi['rate'] = fi['imp'] / fi['imp'].sum()
fi

,feature,imp,rate
0,diff_iq_day,831,0.013850
1,diff_iq_hour,2609,0.043483
2,freq,332,0.005533
3,gender,221,0.003683
4,hour,1487,0.024783
...,...,...,...
167,uf_c2_count,130,0.002167
168,uf_c3_count,275,0.004583
169,uf_c4_count,249,0.004150
170,uf_c5_count,0,0.000000


In [2]:
fi.sort_values(by='rate', ascending=False)[-30:]

NameError: name 'fi' is not defined